# Classe ECDH
A classe ECDH tem como objetivo encapsular o comportamento/dados necessários para implementar o método Diffie-Hellman de troca de chaves. Para tal, são utilizadas as seguintes variáveis:

- `n`: dimensão do corpo base K
- `p = 2`: parâmetro fixo e igual a 2, utilizado na geração de um corpo finito binário.
- `q = p^n`: ordem do corpo finito binário.
- `K = GF(q)`: corpo finito binário.

As restantes variáveis `b` e `G` são inicializadas através da função `setup`, utilizando o seguinte algoritmo:

1. Gera-se aleatoriamente o parâmetro `b`.
2. Gera-se a curva elíptica `E` através do corpo base `K` e do polinómio representado pelos coeficientes [1,1,0,0,b].
3. Calcula-se a ordem da curva (`order_E`).
4. Calcula-se os fatores primos de `order_E`.
5. Caso o maior fator primo seja menor que 2^(n-1), voltar ao ponto 1.
6. Gerar um ponto aleatório (`Q`) da curva elíptica `E`.
7. Calcular a sua ordem (`order_Q`).
8. Se `order_Q` for menor que `order_E`, então volta-se ao ponto 6.
9. Se `order_Q` não for múltiplo de `order_E`, então volta-se ao ponto 6.
10. Calcular o ponto gerador `G` através do ponto `Q`.
11. Retornar o `b` e o `Q`.

Desta forma, dá-se por terminada a fase de setup de todos os parâmetros necessários para a execução do ECDH. 

Além disso, a classe oferece ainda duas funções: `gen_key_pair` e `gen_shared_key`. 

A primeira é responsável por gerar as chaves privada ae pública que serão usadas mais adiante. A chave privada é gerada aleatoriamente do conjunto multiplicativo de Zn. De seguida, calcula-se a chave pública através da multiplicação da chave privada pelo gerador `G`.

Por fim, a função `gen_shared_key` calcula a chave partilhada através da multiplicação da chave privada pela chave pública (ambas recebidas por argumento).

In [1]:
class ECDH:
    def __init__(self, n):
        self.n = n
        self.p = 2
        self.q = self.p^n
        self.K = GF(self.q)
        self.b, self.G = self.setup()

    def setup(self):
        isGenerator = False
        while true:
            b = self.K.random_element()

            E = EllipticCurve(self.K, [1,1,0,0,b])
            order_E = E.order()
            prime_factors = [ f for (f,_) in factor(order_E) ]
            highest_prime_factor = prime_factors[-1]

            if highest_prime_factor >= 2^(self.n - 1):

                for i in range(0, 1000):
                    Q = E.random_element()
                    order_Q = Q.order()
                    
                    if order_Q >= highest_prime_factor and \
                       order_Q % highest_prime_factor == 0:
                        isGenerator = True
                        break
                        
                if isGenerator:
                    G = (order_E//highest_prime_factor) * Q
                    break
        return b, G

    def gen_key_pair(self):
        N = self.G.order()
        private_key = ZZ.random_element(1, N)
        public_key = private_key * self.G
        return private_key, public_key
    
    def gen_shared_key(self, private_key, peer_public_key):
        return private_key * peer_public_key

# Setup

Nesta fase é realizado o *setup* do ECDH como descrito anteriormente. De facto, esta componente encontra-se separada do resto do exemplo de utilização uma vez que pode ser reutilizada em diferentes execuções.

In [2]:
ecdh = ECDH(163)

# Exemplo de utilização da classe ECDH

Primeiramente são gerados os pares de chaves para o agente A e B. De seguida, apesar de não ser explícito deve existir uma troca de chaves públicas entre o A e o B. Posteriormente, cada um dos agentes utiliza a sua chave privada e a chave pública recebida para gerar a chave partilhada. Por fim, o teste `A_shared_key == B_shared_key` dando verdadeiro confirma que ambos os agentes obtiveram o mesmo segredo, ou seja, o ECDH foi realizado com sucesso.

In [3]:
A_private_key, A_public_key = ecdh.gen_key_pair()
B_private_key, B_public_key = ecdh.gen_key_pair()

A_shared_key = ecdh.gen_shared_key(A_private_key, B_public_key)
B_shared_key = ecdh.gen_shared_key(B_private_key, A_public_key)

print A_shared_key == B_shared_key

True
